# CNN과 전이학습

치매환자와 일반인의 뇌 예측

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers

import numpy as np
import matplotlib.pyplot as plt

# !git clone https://github.com/taehojo/data-ch20.git

# 학습셋의 변형을 설정
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # 픽셀의 값을 0.0~1.0사이로 정규화합니다.
    horizontal_flip=True,  # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
    width_shift_range=0.1,  # 전체 크기의 10% 범위에서 좌우로 이동합니다.
    height_shift_range=0.1,
)  # 마찬가지로 위, 아래로 이동합니다.

train_generator = train_datagen.flow_from_directory(
    "./data-ch20/train", target_size=(150, 150), batch_size=5, class_mode="binary"  # 학습셋이 있는 폴더의 위치
)

# 테스트셋은 이미지 부풀리기 과정을 진행하지 않음
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    "./data-ch20/test", target_size=(150, 150), batch_size=5, class_mode="binary"  # 테스트셋이 있는 폴더의 위치
)

# 앞서 배운 CNN 모델을 만들어 적용
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(150, 150, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.summary()

# 모델의 실행 옵션을 설정
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# 모델 최적화를 위한 설정
# modelpath = "./data-ch20/MNIST_CNN.hdf5"
modelpath = "./data-ch20/MNIST_CNN.keras"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor="val_loss", verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=10)

# 모델을 실행
history = model.fit(train_generator, validation_data=test_generator, epochs=30, verbose=1, callbacks=[early_stopping_callback, checkpointer])

# 테스트 정확도를 출력.
print("\n Test Accuracy: %.4f" % (model.evaluate(test_generator)[1]))

fatal: destination path 'data-ch20' already exists and is not an empty directory.
Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 72, 72, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 18496)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │     1,183,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,212,513 (4.63 MB)

 Trainable params: 1,212,513 (4.63 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30


/home/kunsu/miniconda3/envs/fast/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5371 - loss: 0.8388
Epoch 1: val_loss improved from inf to 0.69180, saving model to ./data-ch20/MNIST_CNN.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5349 - loss: 0.8332 - val_accuracy: 0.5750 - val_loss: 0.6918
Epoch 2/30
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5644 - loss: 0.6923
Epoch 2: val_loss improved from 0.69180 to 0.68493, saving model to ./data-ch20/MNIST_CNN.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5664 - loss: 0.6921 - val_accuracy: 0.6583 - val_loss: 0.6849
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5191 - loss: 0.6918
Epoch 3: val_loss did not improve from 0.68493
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5191 - loss: 0.6919 - val_accuracy: 0.5000 - val_loss: 0.6915
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4034 - loss: 0.7029
Epoch 4: val_loss did not improve from 0.68493
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step -

# 전이학습 진행하기

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

# 학습셋의 변형을 설정
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # 주어진 이미지의 크기를 설정
    horizontal_flip=True,  # 수평 대칭 이미지를 50% 확률로 만들어 추가
    width_shift_range=0.1,  # 전체 크기의 10% 범위에서 좌우로 이동
    height_shift_range=0.1,
)  # 마찬가지로 위, 아래로 이동

train_generator = train_datagen.flow_from_directory("./data-ch20/train", target_size=(150, 150), batch_size=5, class_mode="binary")

# 테스트셋의 정규화를 설정
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory("./data-ch20/test", target_size=(150, 150), batch_size=5, class_mode="binary")

# VGG16 모델을 로드
transfer_model = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
transfer_model.trainable = False

# 자체 모델을 설정
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation="relu"))
finetune_model.add(Dropout(0.5))
finetune_model.add(Dense(1, activation="sigmoid"))

# 모델의 실행 옵션을 설정
finetune_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# 학습의 조기 중단을 설정
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=10)

# 모델을 실행
history = finetune_model.fit(train_generator, epochs=30, validation_data=test_generator, validation_steps=10, callbacks=[early_stopping_callback])

# 테스트 정확도를 출력
print("\n Test Accuracy: %.4f" % (finetune_model.evaluate(test_generator)[1]))

Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.5459 - loss: 0.8407 - val_accuracy: 0.8000 - val_loss: 0.5265
Epoch 2/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.7158 - loss: 0.5399 - val_accuracy: 0.8000 - val_loss: 0.4827
Epoch 3/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.8377 - loss: 0.4372 - val_accuracy: 0.9600 - val_loss: 0.3193
Epoch 4/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.8589 - loss: 0.3761 - val_accuracy: 0.9000 - val_loss: 0.2888
Epoch 5/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.8526 - loss: 0.3345 - val_accuracy: 0.8400 - val_loss: 0.3790
Epoch 6/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.8768 - loss: 0.2970 - val_accuracy: 0.7200 - val_loss: 0.5285
Epoch 7/30
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - accuracy: 0.7511 - loss: 0.4737 - val_accuracy: 0.92